In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install  bitsandbytes

In [6]:
!pip install python-dotenv

In [9]:
!pip install PyPDF2

     ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
     - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
     - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
     ------ ------------------------------ 41.0/232.6 kB 245.8 kB/s eta 0:00:01
     ------------- ----------------------- 81.9/232.6 kB 416.7 kB/s eta 0:00:01
     -------------------------- --------- 174.1/232.6 kB 655.4 kB/s eta 0:00:01
     ---------------------------------- - 225.3/232.6 kB 724.0 kB/s eta 0:00:01
     ---------------------------------- - 225.3/232.6 kB 724.0 kB/s eta 0:00:01
     ------------------------------------ 232.6/232.6 kB 526.9 kB/s eta 0:00:00


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import HuggingFaceHub
from langchain.llms import Together

In [3]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [4]:
key = os.getenv("TOGETHER_API_KEY")
print(key)

58d8baed0cc5788ef9c97ce4bad9fc8e2ef03db3ba17031b22acbc552a4ba6b7


In [89]:
llm = Together(
    model="togethercomputer/llama-2-70b-chat",
    temperature=0.9,
    max_tokens=250,
    top_k=1,
    
)

In [90]:
llm

Together(together_api_key=SecretStr('**********'), model='togethercomputer/llama-2-70b-chat', temperature=0.9, top_k=1, max_tokens=250)

In [91]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [92]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [93]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [94]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [95]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [96]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [97]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [98]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [99]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [100]:
TEXT = """Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, biologists use the scientific method to make observations, pose questions, generate hypotheses, perform experiments, and form conclusions about the world around them.[1]

Life on Earth, which emerged more than 3.7 billion years ago,[10] is immensely diverse. Biologists have sought to study and classify the various forms of life, from prokaryotic organisms such as archaea and bacteria to eukaryotic organisms such as protists, fungi, plants, and animals. These various organisms contribute to the biodiversity of an ecosystem, where they play specialized roles in the cycling of nutrients and energy through their biophysical environment."""

In [101]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [102]:
NUMBER=3
SUBJECT="biology"
TONE="simple"

In [103]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [104]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0


In [105]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [106]:
quiz=response.get("quiz")
print(quiz)

### RESPONSE
{
"1": {
"mcq": "What is the scientific study of life?",
"options": {
"a": "Biology",
"b": "Physics",
"c": "Chemistry",
"d": "Mathematics"
},
"correct": "a"
},
"2": {
"mcq": "What is the unity and diversity of life explained by?",
"options": {
"a": "Evolution",
"b": "Reproduction",
"c": "Growth",
"d": "Movement"
},
"correct": "a"
},
"3": {
"mcq": "What is the biodiversity of an ecosystem contributed by?",
"options": {
"a": "Prokaryotic organisms",
"b": "Eukaryotic organisms",
"c": "Both a and b",
"d": "None of the above"
},
"correct": "c"
}
}

Please provide your response in the


In [110]:
# Find the index of the opening curly brace
start_index = quiz.find('{')

# Find the index of the closing curly brace
end_index = quiz.rfind('}')

# Extract the JSON string from the original string
json_string = quiz[start_index:end_index + 1]

# Parse the JSON string into a dictionary
quiz = json.loads(json_string)

# Now 'response' contains the dictionary without '### RESPONSE'
print(quiz)




{'1': {'mcq': 'What is the scientific study of life?', 'options': {'a': 'Biology', 'b': 'Physics', 'c': 'Chemistry', 'd': 'Mathematics'}, 'correct': 'a'}, '2': {'mcq': 'What is the unity and diversity of life explained by?', 'options': {'a': 'Evolution', 'b': 'Reproduction', 'c': 'Growth', 'd': 'Movement'}, 'correct': 'a'}, '3': {'mcq': 'What is the biodiversity of an ecosystem contributed by?', 'options': {'a': 'Prokaryotic organisms', 'b': 'Eukaryotic organisms', 'c': 'Both a and b', 'd': 'None of the above'}, 'correct': 'c'}}


In [111]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


In [112]:
quiz_table_data

[{'MCQ': 'What is the scientific study of life?',
  'Choices': 'a: Biology | b: Physics | c: Chemistry | d: Mathematics',
  'Correct': 'a'},
 {'MCQ': 'What is the unity and diversity of life explained by?',
  'Choices': 'a: Evolution | b: Reproduction | c: Growth | d: Movement',
  'Correct': 'a'},
 {'MCQ': 'What is the biodiversity of an ecosystem contributed by?',
  'Choices': 'a: Prokaryotic organisms | b: Eukaryotic organisms | c: Both a and b | d: None of the above',
  'Correct': 'c'}]

In [113]:
quiz=pd.DataFrame(quiz_table_data)

In [114]:
quiz

,MCQ,Choices,Correct
0,What is the scientific study of life?,a: Biology | b: Physics | c: Chemistry | d: Ma...,a
1,What is the unity and diversity of life explai...,a: Evolution | b: Reproduction | c: Growth | d...,a
2,What is the biodiversity of an ecosystem contr...,a: Prokaryotic organisms | b: Eukaryotic organ...,c


In [117]:
quiz.shape

(3, 3)

In [116]:
quiz.to_csv("quiz.csv",index=False)